if required install libraries as below.

In [18]:
# Install from terminal:
#pip install openpyxl
#pip install pandas==2.0.0
#pip install numpy
#pip install fuzzywuzzy

In [19]:
import openpyxl
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

The Input Files and sheet names

In [20]:
ODM_ID = 427392

AVL_name="BOM_AVL_" + str(ODM_ID) +".xlsx"
AVL_sheet_name="Tabelle1"

Manf_name="Manufacturer_" + str(ODM_ID)+ ".xlsx"
Manf_sheet_name="Supplier"

Please do not make changes below this line unless required to do

Read Input Files, get odm ID, create dataframes

In [21]:
workbook = openpyxl.load_workbook("Input_Files/"+AVL_name)
worksheet = workbook[AVL_sheet_name]
get_odm_id = worksheet.cell(2,2).value
print(f"The ODM ID: {get_odm_id}")

for cell in worksheet["Z"]:
    cell.value = None

workbook_manf = openpyxl.load_workbook("Input_Files/"+Manf_name)
worksheet_manf = workbook_manf[Manf_sheet_name]

for cell in worksheet_manf["P"]:
    cell.value = None


df = pd.read_excel("Input_Files/"+AVL_name, sheet_name=AVL_sheet_name)
df_manf = pd.read_excel("Input_Files/"+Manf_name, sheet_name=Manf_sheet_name)

errors = []


The ODM ID: 427392


Check ODM_Component_Name if the lenght is more than 128 chars

In [22]:
def component_name():
    limit = False

    for i, value in df["ODM_Component_Name"].items():
        if len(str(value)) > 128:
            limit = True
            error = f"WARNING: ODM_Component_Name is more than 128 chars. Row: {i+2}, Sheet: AVL"
            errors.append(error)

    
    if not limit:
        print("TRUE: ODM_Component_Name:No lenght is more than 128 chars")

component_name()


TRUE: ODM_Component_Name:No lenght is more than 128 chars


Check the Order_Code, if its blank, short or too long

In [23]:
df_blacklist_order_code = pd.read_excel("Required_Mapping_Files/ODM_Import_Config_NOT_Relevant_ManufacturerParts_List.xlsx")
blacklist = df_blacklist_order_code["blacklisted ordercodes"]

def order_code():
    limit = False
    for i,value in df["Order_Code"].items():
        
        if value in blacklist:
            limit = True
            error = f"WARNING: There are blacklisted Order_Code. Row: {i+2}, Sheet: AVL"
            errors.append(error)

        if pd.isnull(value):
            limit = True
            error = f"ERROR: There are blank Order_Code. Row: {i+2}, Sheet: AVL"
            errors.append(error)

        elif len(str(value)) < 3:
            limit = True
            error = f"ERROR: Value at Order_Code is short. Character count: {len(str(value))}. Row: {i+2}, Sheet: AVL"
            errors.append(error)

        elif len(str(value)) > 40:
            limit = True
            error= f"ERROR: Value at Order_Code is long. Character count: {len(str(value))}. Row: {i+2}, Sheet: AVL"
            errors.append(error)

    if not limit:
        print("TRUE: Order_Code column has no blank, short or long values")


order_code()

Check if column D and F have the same values

In [24]:
def check_column_values():
    column1 = df["BSH_SAP_Material_Number"].iloc[1:]
    column2 = df["BSH_PCBA_SAP Material_Number"].iloc[1:]

    if any(value1 != value2 for value1, value2 in zip(column1, column2)):
        warning= "WARNING: There are different values in column D and F.Check Manually"
        errors.append(warning)
    else:       
        warning = "WARNING: The values in column D and F are the same. Remove the values in F"
        errors.append(warning)

check_column_values()

Check unit of measure

In [25]:
plmp_df = pd.read_excel("Required_Mapping_Files/"+"UoM-naming-TC-with-description.xlsx", usecols=["Abbreviation"])
odm_df = pd.read_excel("Required_Mapping_Files/"+"ODM_Import_Config_UoM_Mapping.xlsx", usecols=["ODM UoM"])

def check_uom():
        
    errors_found = False
    uom_column = df["UoM"].str.replace(" ", "")
    quantity_column = pd.to_numeric(df["Quantity"])

    for i, uom in enumerate(uom_column, start=1):
        if pd.isnull(uom):
            error = f"ERROR: There are blank UoM. Row: {i+1}, Sheet: AVL"
            errors.append(error)
            errors_found = True
        else:
            uom = uom.lower()
            
        if np.logical_not((uom == plmp_df["Abbreviation"].str.lower().values).any() or (uom == odm_df["ODM UoM"].str.lower().values).any()):
            error = f"ERROR: There is an error with UoM. Row: {i+1}, Sheet: AVL"
            errors.append(error)
            errors_found = True

        if i > 1 and uom in ["pcs", "pc"]:
            quantity = quantity_column.iloc[i-1]
            if pd.isnull(quantity) or not quantity.is_integer():
                error = f"ERROR: A PC/PCS and non-integer value matches. Row: {i+1}, Sheet: AVL"
                errors.append(error)
                errors_found = True
    
    if not errors_found:
        print("TRUE: Usage of Unit of Measure is correct")

check_uom()



In [26]:
plmp_df["Abbreviation"].str.lower().values

array(['as required', 'bag', 'box', 'ccm', 'cd3', 'cm', 'cm2', 'cl', 'dm',
       'ca', 'dz', 'ft', 'ft2', 'ft3', 'g', 'gal', 'in', 'can', 'car',
       'kg', 'km', 'kt', 'l', 'lb', 'm', 'm2', 'm3', 'um', 'ml', 'mm',
       'mm2', 'mm3', 'oz', 'foz', 'pac', 'pal', 'pq', 'qt', 'rol', 'kit',
       'set', 'pc'], dtype=object)

Check Manf_Ids for blank and unquie ID usage

Both in AVL an Manf_List

In [27]:
def check_Manf_ID_AVL():
    manf_dict = {}
    errors_found = False

    for i, row in df.iterrows():
        manf_id = row['Manufacturer_ID']
        manf_name = row['Manufacturer_Name'].lower()
        
        # Check for blank Manf_ID values
        if pd.isnull(manf_id):
            error = f"ERROR_AVL: There is a blank Manufacturer_ID. Row: {i+2}, Sheet: AVL"
            errors.append(error)
            errors_found = True
        
        # Check for matching Manf_ID and Manf_Name pairs
        elif manf_id in manf_dict:
            if manf_dict[manf_id] != manf_name:
                error = f"ERROR_AVL: The same Manufacturer_ID with different Manufacturer_Name matching issue. Row: {i+2}, Sheet: AVL"
                errors.append(error)
                errors_found = True

        else:
            # Check for different ID for the same Manf_Name
            if manf_name in manf_dict.values():
                error = f"ERROR_AVL: The Same Manufacturer_Name with different Manufacturer_ID matching issue. Row: {i+2}, Sheet: AVL"
                errors.append(error)
                errors_found = True
    
            else:
                manf_dict[manf_id] = manf_name

    if not errors_found:
        print("TRUE_AVL: No Manf_Id errors found")


def check_Manf_ID_manf():
    manf_dict = {}
    errors_found = False

    for i, row in df_manf.iterrows():
        manf_id = row['ODM_Manufacturer_Distributor_ID']
        manf_name = row['ODM_Manufacturer_Distributor_Name'].lower()
        
        # Check for blank Manf_ID values
        if pd.isnull(manf_id):
            error = f"ERROR_Manf: There is a blank Manufacturer_ID. Row: {i+2}, Sheet: Manf"
            errors.append(error)
            errors_found = True
        
        # Check for matching Manf_ID and Manf_Name pairs
        elif manf_id in manf_dict:
            if manf_dict[manf_id] != manf_name:
                error = f"ERROR_Manf: The same Manufacturer_ID with different Manufacturer_Name matching issue. Row: {i+2}, Sheet: Manf"
                errors.append(error)
                errors_found = True        
    
        else:
            # Check for different ID for the same Manf_Name
            if manf_name in manf_dict.values():
                error = f"ERROR_Manf: The Same Manufacturer_Name with different Manufacturer_ID matching issue. Row: {i+2}, Sheet: Manf"
                errors.append(error)
                errors_found = True
            else:
                manf_dict[manf_id] = manf_name

    if not errors_found:
        print("TRUE_Manf: No Manf_Id errors found")

check_Manf_ID_AVL()
check_Manf_ID_manf()


TRUE_AVL: No Manf_Id errors found
TRUE_Manf: No Manf_Id errors found


Check latin chars in entire files 

Both in AVL and Manf_List

In [28]:
skip_characters = ["Ö", "Ü", "Ä", "µ", "ß", "@", "$", "%","ü","ö","ä",",","–","+","*","Ω","º","，","č","Š","°"]

def check_AVL_latin_characters():
    errors_found = False
    for column in df.columns:
        if column != "ODM_Component_Name":

            for i, value in enumerate(df[column]):
                if isinstance(value, str) and all(char not in value for char in skip_characters):
                    value = value.replace(" ", "")  # Remove spaces in the value
                    if not value.isascii():
                        error =f"ERROR_AVL: Use of non-Latin characters in column '{column}'. Row: {i+2}, Sheet: AVL"
                        errors.append(error)
                        errors_found = True

                    
    if not errors_found:
        print("True useage of latin characters in AVL")

def check_Manf_latin_characters():
    errors_found = False
    for column in df_manf.columns:
        if column != "Street_And_House_Number" and column != "City":
            for i, value in enumerate(df_manf[column]):
                if isinstance(value, str) and all(char not in value for char in skip_characters):
                    value = value.replace(" ", "")  # Remove spaces in the value
                    if not value.isascii():
                        error =f"ERROR_Manf: Use of non-Latin characters in column '{column}'. Row: {i+2}, Sheet: Manf"
                        errors.append(error)
                        errors_found = True


    if not errors_found:
        print("TRUE: Usage of latin characters in Manufacturer List are correct")

check_AVL_latin_characters()
check_Manf_latin_characters()

TRUE: Usage of latin characters in Manufacturer List are correct


Check Country Codes in Manf_Sheet

In [29]:
df_county_codes = pd.read_excel("Required_Mapping_Files/"+"countrycodes.xlsx")

def check_county_codes():
    errors_found = False

    column1 = df_county_codes["code"]
    column2 = df_manf["Country"]

    for i, value in enumerate(column2,start=2):
        
        if pd.isnull(value):
            error = f"WARNING: There are blank Coutry Code. Row: {i}, Sheet: Manf"
            errors.append(error)
            errors_found = True

        if value not in column1.values and pd.notnull(value):
            error= f"ERROR: There is wrong use of country codes. Row: {i}, Sheet: Manf"
            errors.append(error)
            errors_found = True

    if not errors_found: 
        print("TRUE: The use of country codes are true.")


check_county_codes()

Check Order Code release status

In [30]:
df_release_status = pd.read_excel("Required_Mapping_Files/"+"ODM_Import_Config_Status_Mapping.xlsx")

def check_release_status():
    errors_found = False

    column1 = df_release_status[df_release_status.iloc[:, 0] == get_odm_id].iloc[:, 1:7].values.flatten()
    column1 = [item.lower() for sublist in column1 if pd.notnull(sublist) for item in sublist.split(";") if item.strip()]
    column2 = [header.lower() for header in df_release_status.columns[1:]]  # Get lowercase values of headers from column 2 onwards
    column1.extend(column2)  # Merge column2 with column1
    print(column1)
    
    column2 = df["Order_Code_Release_Status"].str.lower()

    for i, value in enumerate(column2,start=2):
            if pd.isnull(value):
                error= f"ERROR: The release status is blank. Row: {i}, Sheet: AVL"
                errors.append(error)
                errors_found = True
  
            else:        
                if value not in column1:
                    error= f"ERROR: The release status is wrong or not included. Row: {i}, Sheet: AVL"
                    errors.append(error)
                    errors_found = True

    if not errors_found: 
        print("TRUE: The use of release status are true.")


check_release_status()

['new', 'component meeting', 'prerelease', 'released', 'disqualified', 'phaseout', 'check', 'unnamed: 8', 'unnamed: 9', 'unnamed: 10', 'unnamed: 11', 'unnamed: 12']
TRUE: The use of release status are true.


Check blank or 0 quantity - ignore if the name includes packaging material


In [31]:
def check_quantity():
    errors_found = False

    column1 = df["Quantity"]
    column2 = df["ODM_Component_Name"]

    for i, value in enumerate(column1,start=2):
       
        if pd.isnull(value):
            error = f"ERROR: There are blank Quantity. Row: {i}, Sheet: AVL"
            errors.append(error)
            errors_found = True

        if value == 0 and any(keyword in column2[i-2].lower() for keyword in ["pack", "label", "sticker", "pallet", "palette", "etikett"]):
            error= f"WARNING: Packaging related material quantity is 0. Row: {i}, Sheet: AVL"
            errors.append(error)
            errors_found = True
        else:
            if value == 0:
                error= f"ERROR: The quantity is 0. Row: {i}, Sheet: AVL"
                errors.append(error)
                errors_found = True

    if not errors_found: 
        print("TRUE: The use of quantity codes are true.")


check_quantity()

TRUE: The use of quantity codes are true.


Similarity Check for Manufacturer Name

In [32]:
def similarty_check (Manf_1, Manf_2):
    return fuzz.ratio(Manf_1.lower(), Manf_2.lower())

duplicates = {}
is_duplicate = False

for i in range(len(df_manf["ODM_Manufacturer_Distributor_Name"])):
    manuf = df_manf["ODM_Manufacturer_Distributor_Name"][i]
    for j in range (i+1, len(df_manf["ODM_Manufacturer_Distributor_Name"])):
        if similarty_check(manuf, df_manf["ODM_Manufacturer_Distributor_Name"][j]) > 90:
            is_duplicate = True
            error= f"WARNING: This Manufacturer_Name may have a duplicate in {j+2}. Row: {i+2}, Sheet: Manf"
            errors.append(error)
            print(error)

if not is_duplicate: 
    print("TRUE: There are not duplicates.")


TRUE: There are not duplicates.


Print and Save the errors to excel

In [33]:
workbook_error = openpyxl.load_workbook("Output_Files/errors_excel.xlsx")
sheet = workbook_error["Sheet1"]
sheet.delete_rows(1, sheet.max_row)
workbook_error.save("Output_Files/errors_excel.xlsx")


error_df = pd.DataFrame(errors)
error_df = error_df.rename(columns={error_df.columns[0]: "Error"})
error_df["Row"] = error_df["Error"].str.extract("Row: (\d+)")
error_df["Sheet"] = error_df["Error"].str.extract("Sheet: (\w+)")
error_df.to_excel("Output_Files/errors_excel.xlsx")
pd.set_option("display.max_colwidth", 200)
error_df.index = error_df.index + 1
error_df

Error   
1    ERROR: Value at Order_Code is long. Character count: 52. Row: 384, Sheet: AVL  \
2    ERROR: Value at Order_Code is long. Character count: 52. Row: 734, Sheet: AVL   
3    ERROR: Value at Order_Code is long. Character count: 52. Row: 807, Sheet: AVL   
4    ERROR: Value at Order_Code is long. Character count: 52. Row: 884, Sheet: AVL   
5    ERROR: Value at Order_Code is long. Character count: 52. Row: 944, Sheet: AVL   
..                                                                             ...   
148                     WARNING: There are blank Coutry Code. Row: 76, Sheet: Manf   
149                     WARNING: There are blank Coutry Code. Row: 77, Sheet: Manf   
150                     WARNING: There are blank Coutry Code. Row: 78, Sheet: Manf   
151                     WARNING: There are blank Coutry Code. Row: 79, Sheet: Manf   
152                     WARNING: There are blank Coutry Code. Row: 80, Sheet: Manf   

     Row Sheet  
1    384   AVL  
2    734   AVL  
3    807   AVL  
4    884   AVL  
5    944   AVL  
..   ...   ...  
148   76  Manf  
149   77  Manf  
150   78  Manf  
151   79  Manf  
152   80  Manf  

[152 rows x 3 columns]

Write errors to AVL and Manf_List like Teamcenter

In [34]:
for i, row in error_df.iterrows():
    value = row["Sheet"]
    error = row["Error"]
    row_no = row["Row"]

    if value == "AVL" and pd.notnull(row_no):
        existing_error = worksheet[f"Z{row_no}"].value
        if existing_error:
            error = f"{existing_error} - {error}"
        worksheet[f"Z{row_no}"] = error  # Write error to column Z in AVL worksheet

    if value == "Manf" and pd.notnull(row_no):
        existing_error = worksheet_manf[f"P{row_no}"].value
        if existing_error:
            error = f"{existing_error} - {error}"
        worksheet_manf[f"P{row_no}"] = error  # Write error to column P in Manf worksheet

workbook.save("Output_Files/"+AVL_name)
workbook_manf.save("Output_Files/"+Manf_name)
